In [6]:
import pandas as pd
import ast # Library untuk mengubah string menjadi list

# Muat data Anda
df = pd.read_csv('hasil_preprocessing_manajemen.csv')

# Ubah kolom 'hasil_preprocessing' dari string kembali ke list
df['hasil_preprocessing'] = df['hasil_preprocessing'].apply(ast.literal_eval)

# Tampilkan untuk memastikan formatnya benar
print(df.head())

# Siapkan data teks untuk model
kalimat = df['hasil_preprocessing'].tolist()

                                          abstrak_id  \
0  ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...   
1  Tujuan penelitian ini adalah untuk mengetahui ...   
2  Aplikasi nyata pemanfaatan teknologi informasi...   
3  Abstrak\r\nPenelitian ini menggunakan metode k...   
4  Abstrak\r\n\r\nAththaariq, Pengaruh Kompetensi...   

                                 hasil_preprocessing  
0  [abstrak, satiyah, pengaruh, faktorfaktor, lat...  
1  [tuju, teliti, persepsi, brand, association, l...  
2  [aplikasi, nyata, manfaat, teknologi, informas...  
3  [abstrak, teliti, metode, kuantitatif, tekan, ...  
4  [abstrak, aththaariq, pengaruh, kompetensi, do...  


In [7]:
from gensim.models import Word2Vec

# Latih model Word2Vec
# vector_size=100 -> Ini adalah parameter kunci sesuai tugas Anda
# window=5 -> Jarak kata tetangga yang dipertimbangkan
# min_count=2 -> Abaikan kata yang muncul kurang dari 2 kali
# workers=4 -> Jumlah thread CPU yang digunakan (sesuaikan dengan komputer Anda)
model_w2v = Word2Vec(sentences=kalimat, vector_size=100, window=5, min_count=2, workers=4)

print("Model Word2Vec berhasil dilatih!")

Model Word2Vec berhasil dilatih!


In [13]:
import numpy as np
from sklearn.cluster import KMeans

# Fungsi untuk membuat vektor dokumen dengan merata-ratakan word vector
def create_document_vector(doc, model):
    # Dapatkan semua vektor kata yang ada di model
    word_vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Buat daftar vektor untuk semua dokumen
X = [create_document_vector(doc, model_w2v) for doc in kalimat]

# --- Proses Clustering ---
# Ubah n_clusters menjadi jumlah topik yang Anda inginkan (misal: 3)
jumlah_topik = 5
kmeans = KMeans(n_clusters=jumlah_topik, random_state=42, n_init=10)
kmeans.fit(X)

# Tambahkan label cluster (topik) ke DataFrame utama Anda
df['topik'] = kmeans.labels_

# --- TAMBAHKAN KODE INI UNTUK MENYIMPAN ---
df.to_csv('hasil_topic_modeling.csv', index=False)

print("DataFrame dengan label topik berhasil disimpan ke 'hasil_topic_modeling.csv'")
print(df.head())

DataFrame dengan label topik berhasil disimpan ke 'hasil_topic_modeling.csv'
                                          abstrak_id  \
0  ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...   
1  Tujuan penelitian ini adalah untuk mengetahui ...   
2  Aplikasi nyata pemanfaatan teknologi informasi...   
3  Abstrak\r\nPenelitian ini menggunakan metode k...   
4  Abstrak\r\n\r\nAththaariq, Pengaruh Kompetensi...   

                                 hasil_preprocessing  topik  
0  [abstrak, satiyah, pengaruh, faktorfaktor, lat...      2  
1  [tuju, teliti, persepsi, brand, association, l...      3  
2  [aplikasi, nyata, manfaat, teknologi, informas...      3  
3  [abstrak, teliti, metode, kuantitatif, tekan, ...      2  
4  [abstrak, aththaariq, pengaruh, kompetensi, do...      2  


In [ ]:
# Tampilkan 5 dokumen pertama untuk setiap topik
for i in range(jumlah_topik):
    print(f"\\n--- Topik {i} ---")
    # Ambil 5 judul dari abstrak yang masuk ke topik ini
    contoh_dokumen = df[df['topik'] == i]['abstrak_id'].head(5).tolist()
    for idx, abstrak in enumerate(contoh_dokumen):
        # Tampilkan 100 karakter pertama dari setiap abstrak
        print(f"{idx+1}. {abstrak[:100]}...")

\n--- Topik 0 ---
1. ABSTRAK
Tujuan penelitian ini adalah untuk mengetahui pengaruh variabel current ratio, return on eq...
2. Pendekatan penelitian yang digunakan dalam penelitian ini adalah pendekatan kuantitatif. Populasi pe...
3. Devi Kurniawati, Pengaruh Earning Per Share, Net Profit Margin, Debt to Asset Ratio dan Return On eq...
4. Tujuan dari penelitian ini adalah untuk mengetahui pengaruh rasio profitabilitas terhadap harga saha...
5. Abstrak

Pendekatan penelitian yang digunakan dalam penelitian ini adalah pendekatan kuantitatif. ...
\n--- Topik 1 ---
1. ABSTRAK
Haryono Arifin, Pengaruh Perilaku Konsumen Terhadap Keputusan Pembelian Produk Honda Merek ...
2. ABSTRAK

Tujuan penelitian ini adalah untuk mengidentifikasi variabel-variabel yang mempengaruhi k...
3. Tujuan penelitian ini adalah untuk mengetahui pengaruh variabel Retailing Mix yang terdiri dari loka...
4. Tujuan penelitian ini adalah mengetahui pengaruh celebrity endorser Iwan Fals terhadap minat beli pr...
5. Baur

: 